# Google Tasks API exploration

Use this notebook to authenticate with Google, cache the credentials, and test the Tasks endpoints for lists and tasks.


## Setup checklist
1. Create a Google Cloud project at https://console.cloud.google.com/.
2. Enable the **Tasks API** and create an OAuth 2.0 Client ID (Desktop application).
3. Download the `client_secret_<id>.json` file and set `GOOGLE_CLIENT_SECRETS_PATH` to point to it (or paste its JSON into `GOOGLE_CLIENT_SECRETS`).
4. Install the dependencies (next cell) and rerun this notebook so the auth cell can run.

**Storage notes**
- Credentials are cached in `.google_tasks_token.json` by default; you can override with `GOOGLE_TASKS_TOKEN`.
- Use scopes that match your needs (`https://www.googleapis.com/auth/tasks` for full CRUD).


In [ ]:
# install requirements if you haven't already
# !pip install --upgrade google-auth google-auth-oauthlib google-api-python-client

In [ ]:
import json
import os
from pathlib import Path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/tasks']
CLIENT_SECRETS_PATH = os.environ.get('GOOGLE_CLIENT_SECRETS_PATH', '').strip()
CLIENT_SECRETS_JSON = os.environ.get('GOOGLE_CLIENT_SECRETS')
TOKEN_PATH = Path(os.environ.get('GOOGLE_TASKS_TOKEN', '.google_tasks_token.json'))

if not CLIENT_SECRETS_PATH and not CLIENT_SECRETS_JSON:
    raise ValueError('Set GOOGLE_CLIENT_SECRETS_PATH to the downloaded JSON or provide GOOGLE_CLIENT_SECRETS.')

if CLIENT_SECRETS_JSON:
    client_config = json.loads(CLIENT_SECRETS_JSON)
else:
    client_config = json.loads(Path(CLIENT_SECRETS_PATH).read_text())


In [ ]:
def save_credentials(creds):
    TOKEN_PATH.write_text(creds.to_json())

def get_credentials():
    creds = None
    if TOKEN_PATH.exists():
        creds = Credentials.from_authorized_user_file(str(TOKEN_PATH), SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_config(client_config, SCOPES)
            creds = flow.run_console()
        save_credentials(creds)
    return creds

creds = get_credentials()
service = build('tasks', 'v1', credentials=creds)


In [ ]:
def list_tasklists(max_results=50):
    resp = service.tasklists().list(maxResults=max_results).execute()
    return resp.get('items', [])

def insert_tasklist(title):
    return service.tasklists().insert(body={'title': title}).execute()

def list_tasks(tasklist_id, max_results=50):
    resp = service.tasks().list(tasklist=tasklist_id, maxResults=max_results).execute()
    return resp.get('items', [])

def create_task(tasklist_id, title, notes=None):
    body = {'title': title}
    if notes:
        body['notes'] = notes
    return service.tasks().insert(tasklist=tasklist_id, body=body).execute()

def update_task(tasklist_id, task_id, **kwargs):
    body = {k: v for k, v in kwargs.items() if v is not None}
    return service.tasks().patch(tasklist=tasklist_id, task=task_id, body=body).execute()

def delete_task(tasklist_id, task_id):
    return service.tasks().delete(tasklist=tasklist_id, task=task_id).execute()


In [ ]:
# Find or create a task list named 'Codex Test List'
tasklists = list_tasklists()
codex_list = next((tl for tl in tasklists if tl['title'] == 'Codex Test List'), None)
if not codex_list:
    codex_list = insert_tasklist('Codex Test List')
codex_list_id = codex_list['id']
codex_list


In [ ]:
# Create a task
created_task = create_task(codex_list_id, 'Test task from notebook', notes='Created via Google Tasks API')
created_task


In [ ]:
# List all tasks in the list
tasks = list_tasks(codex_list_id)
tasks


In [ ]:
# Update the first task if it exists
if tasks:
    updated_task = update_task(codex_list_id, tasks[0]['id'], notes='Notes updated from notebook', status='needsAction')
    updated_task


In [ ]:
# Delete a task (optional)
# if tasks:
#     delete_task(codex_list_id, tasks[0]['id'])
#     print('Deleted', tasks[0]['id'])
pass
